In [1]:
!pip install pretrainedmodels 
!pip install wtfml==0.0.2
!pip install flask-ngrok

In [ ]:
# way to upload a image : endpoint
# way to save the image
# function to make prediction on the image
# show the results
import os
import torch

from flask import Flask
from flask import request
from flask import render_template

import albumentations
import pretrainedmodels

import numpy as np
import torch.nn as nn

from torch.nn import functional as F

from wtfml.data_loaders.image import ClassificationLoader
from wtfml.engine import Engine


app = Flask(__name__)
UPLOAD_FOLDER = "D:/Atom_project/Web_App_melanoma/static"
DEVICE = "cpu"
MODEL = None



class SEResnext50_32x4d(nn.Module):
    def __init__(self, pretrained='imagenet'):
        super(SEResnext50_32x4d, self).__init__()
        self.base_model = pretrainedmodels.__dict__[
            "se_resnext50_32x4d"
        ](pretrained=pretrained)
        self.l0 = nn.Linear(2048, 1)

    def forward(self, image, targets):
        batch_size, _, _, _ = image.shape
        x = self.base_model.features(image)
        x = F.adaptive_avg_pool2d(x, 1)
        x = x.reshape(batch_size, -1)
        out = torch.sigmoid(self.l0(x))
        loss = 0

        return out, loss


def predict(image_path, model):
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)

    test_aug = albumentations.Compose(
        [
            albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True)
        ]
    )

    test_images = [image_path]
    test_targets = [0]

    test_dataset = ClassificationLoader(
        image_paths=test_images,
        targets=test_targets,
        resize=None,
        augmentations=test_aug
    )

    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        #as we have only one image to predict
        batch_size=1,
        shuffle=False,
        num_workers=0
    )

    predictions = Engine.predict(test_loader, model, DEVICE)
    predictions = np.vstack((predictions)).ravel()

    return predictions



@app.route("/", methods= ["GET", "POST"])
def upload_predict():
  if request.method == "POST":
    image_file = request.files["image"]                   # get from the input file in index1.html
    if image_file:
      image_location = os.path.join(
          UPLOAD_FOLDER,
          image_file.filename 
      )
      image_file.save(image_location)
      pred = predict(image_location, MODEL)[0]
      return render_template("index.html", prediction=pred, image_loc= image_file.filename)
  return render_template("index.html", prediction= 0, image_loc=None)


if __name__ == "__main__":
        MODEL = SEResnext50_32x4d(pretrained=None)
        MODEL.load_state_dict(torch.load("D:/Atom_project/Web_App_melanoma/model.bin"))
        #https://pytorch.org/tutorials/beginner/saving_loading_models.html 
        MODEL.to(DEVICE)
        app.run(port=12000, debug=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:12000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Jun/2021 21:49:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2021 21:49:48] "GET /static/css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2021 21:49:48] "GET /static/css/form_upload.css HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2021 21:49:48] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [21/Jun/2021 21:50:17] "GET /?fbclid=IwAR2DH6-IFM7DS5juAbUD_nlXye4aQhdaFKoM9xoqNQhCctC_2hJp2Kru22c HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2021 21:50:17] "GET /static/css/bootstrap.min.css HTTP/1.1" 304 -
127.0.0.1 - - [21/Jun/2021 21:50:17] "GET /static/css/form_upload.css HTTP/1.1" 304 -
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]C:\Users\Mishu\AppData\Local\Programs\Python\Python36\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Plea